In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
import re
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification

%matplotlib inline
# ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_json('./data/dataSet_Culture_06102023-POINT.json')

In [3]:
# Extract the year from 'Analysis Date'
df['year'] = df['Analysis Date'].apply(lambda x: x.split('-')[0])

df['polygon_x'] = df['polygon'].apply(lambda x: x['x'])
df['polygon_y'] = df['polygon'].apply(lambda x: x['y'])
df['month'] = df['Analysis Date'].apply(lambda x: x.split('-')[1])
df['day'] = df['Analysis Date'].apply(lambda x: x.split('-')[2])

df['vegetation'] = (df['indextype'] == 'NDVI') & (df['averagevalue'] >= 0.15)
df = df.drop(['polygon', 'soil_id'] , axis = 1)

# Modify the 'combined' column to include year
df['combined'] = df['polygon_x'].astype(str) + '_' + df['polygon_y'].astype(str) + '_' + df['year'].astype(str)

# Assign unique ID based on the grouped column
df['id'] = df.groupby('combined').ngroup() + 1

# Drop the combined column and other temporary columns
df = df.drop(columns=['combined', 'polygon_x', 'polygon_y', 'year'])

In [4]:
df = df[df['indextype'] == 'NDVI']
df = df.drop(columns=['indextype', 'year contour', 'month', 'day', 'vegetation', 'type_culture_name'])
df = df.rename(columns={'culture_name': 'class'})
df = df.rename(columns={'averagevalue': 'red'})
df = df.rename(columns={'Analysis Date': 'date'})
df['date'] = pd.to_datetime(df['date'])

In [5]:
label_encoder = LabelEncoder() 
  
# Encode labels in column 'class'. 
df['class']= label_encoder.fit_transform(df['class']) 
df['district_name']= label_encoder.fit_transform(df['district_name']) 
df['soil_name']= label_encoder.fit_transform(df['soil_name']) 
df['class'].unique() 

array([ 2, 16, 21,  4, 11, 13,  7, 20, 18,  3,  0,  5, 17,  8, 12, 10, 15,
       14,  6,  1, 19,  9])

In [6]:
# Convert date to multiple columns (year, month, day)
df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month
df['day'] = pd.to_datetime(df['date']).dt.day
df.drop('date', axis=1, inplace=True)

In [7]:
# Pivot table
pivot_df = df.pivot_table(index='id', columns='month', values='red', aggfunc='mean')

# Fill NaN values (assuming you want to fill with zeros, adjust if needed)
pivot_df = pivot_df.fillna(0)

# Rename columns as needed
pivot_df.columns = [f'red_{col}_month' for col in pivot_df.columns]

# Reset the index so 'id' becomes a column
pivot_df = pivot_df.reset_index()

# Assuming each 'id' has a unique 'class', get the 'class' value for each 'id' and add to the pivot dataframe
pivot_df['class'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['class'])
pivot_df['elevation_contour'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['elevation_contour'])
pivot_df['district_name'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['district_name'])
pivot_df['soil_name'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['soil_name'])

# Reordering columns
pivot_df = pivot_df[['red_4_month', 'red_5_month', 'red_6_month', 'red_8_month', 'red_9_month', 'id', 'elevation_contour', 'district_name', 'soil_name', 'class']]


In [8]:
# Splitting the data
X = pivot_df.drop(['id', 'class'], axis=1)  # Features excluding 'id' and 'class'
y = pivot_df['class']  # Target variable

In [9]:
# imputer = KNNImputer(n_neighbors=5)
# X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [10]:
class_counts = y.value_counts()

# Identify the classes with 99 or fewer samples
small_sample_classes = class_counts[class_counts <= 99].index

# Create a mask for these classes
small_sample_mask = y.isin(small_sample_classes)

# Set the values for these classes to 0
y[small_sample_mask] = 0

In [11]:
class_counts = y.value_counts()
single_sample_classes = class_counts[class_counts <= 2].index
filter_mask = ~y.isin(single_sample_classes)
X = X[filter_mask]
y = y[filter_mask]

In [12]:
y = pd.Series(y)
print('Class distribution before undersampling:\n', y.value_counts())


Class distribution before undersampling:
 class
11    231
21    193
4     183
13    163
7     120
0     105
Name: count, dtype: int64


In [13]:
rus = RandomUnderSampler(sampling_strategy='auto')
X_undersampled, y_undersampled = rus.fit_resample(X, y)

In [14]:
y_undersampled = pd.Series(y_undersampled)
print('Class distribution after undersampling:\n', y_undersampled.value_counts())

Class distribution after undersampling:
 class
0     105
4     105
7     105
11    105
13    105
21    105
Name: count, dtype: int64


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_undersampled, y_undersampled, test_size=0.2, stratify=y_undersampled)

In [16]:
# Defining the parameter grid for GradientBoostingClassifier
param_dist = {
               'bootstrap': [True, False],
               'max_depth': [10, 20],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [6, 10, 16],
               'min_samples_split': [1,2 ,3 ,4,5 ,6, 7, 10, 16],
               'n_estimators': [130, 180, 250],
               'oob_score':[True, False],

}

# Instantiating GradientBoostingClassifier
gbm = RandomForestClassifier()

# Setting up RandomizedSearchCV
random_search = RandomizedSearchCV(
    gbm, param_distributions=param_dist, n_iter=1000, scoring='accuracy', 
    cv=9, verbose=1, n_jobs=-1
)

# Fitting the RandomizedSearchCV object to the data
random_search.fit(X_train, y_train)

# Getting the best parameters and the best score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

# Evaluate on the test set
print("Test Set Score:", random_search.score(X_test, y_test))

Fitting 9 folds for each of 1000 candidates, totalling 9000 fits
Best Parameters: {'oob_score': False, 'n_estimators': 180, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}
Best Score: 0.43452380952380953
Test Set Score: 0.3968253968253968


In [17]:
print("Train Set Score:", random_search.score(X_train, y_train))

Train Set Score: 0.748015873015873
